In [ ]:
#실질적으로 입력 해야 하는 부분은 year = 2010 이라고 되어 있는데, 여기서 연도를 바꿔 주시면 됩니다.
#셀은 크게 2개로 나누어져 있고 위에는 정치인 연설 데이터를 이용해서 모델 학습 하는 부분이고 
#아래 데이터는 기업 데이터를 이용하여 검증하는 부분입니다. 

In [1]:
# Standard imports
import tweepy
import csv
import re
import operator 
from collections import Counter
from tweepy import OAuthHandler
import sys
import time
#import urllib2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

In [2]:
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [4]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\indisystem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\indisystem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\indisystem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\indisystem\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
years = 2010

pol_2010_data = pd.read_csv("./Congressmen+Speech/POL_RES/polres_"+str(years)+".csv",encoding="euc-kr")
# pol_2010_data = pd.read_csv("./Congressmen+Speech/POL_RES/polres_"+str(years)+".csv",encoding="euc-kr")

# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

pol_2010_data["textlens"] = 0

for te in range(0,len(pol_2010_data)):
    text = sent_tokenize(pol_2010_data["V2"][te])
    del text[0]
    pol_2010_data["textlens"][te] = len(text)
    for i in text:
        
        sentence = word_tokenize(i) # 단어 토큰화를 수행합니다.
        result = []

        for word in sentence: 
            word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
            if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
                if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                    result.append(word)
                    if word not in vocab:
                        vocab[word] = 0 
                    vocab[word] += 1
        sentences.append(result) 
        
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)

word_to_index = {}
i=0
for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 
        i=i+1
        word_to_index[word] = i
        

vocab_size = 99999
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] 
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
#print(word_to_index)


encoded = []
for s in sentences:
    temp = []
    for w in s:
        try:
            temp.append(word_to_index[w])
        except KeyError:
            continue
                        

    encoded.append(temp)
    
list_result = []
for i in range(0,len(pol_2010_data["textlens"])):
    for j in range(0,pol_2010_data["textlens"][i]):
        list_result.append(pol_2010_data["result"][i])
          
        
print('최대 길이 :{}'.format(max(len(l) for l in encoded)))
print('평균 길이 :{}'.format(sum(map(len, encoded))/len(encoded)))

# 나누기
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(encoded, list_result, test_size=0.3, stratify=list_result)
        
max_len = 30
X_TRAIN = pad_sequences(df_x_train, maxlen=max_len) # 훈련용 패딩
X_TEST = pad_sequences(df_x_test, maxlen=max_len) # 훈련용 패딩
Y_TRAIN = to_categorical(df_y_train)
Y_TEST = to_categorical(df_y_test)

model = Sequential()
model.add(Embedding(100000, 120))
model.add(LSTM(120))
model.add(Dense(2, activation='softmax'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('best_model_'+str(years)+'.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
history = model.fit(X_TRAIN, Y_TRAIN, batch_size=128, epochs=30, callbacks=[es, mc], validation_data=(X_TEST, Y_TEST))

loaded_model = load_model('best_model_'+str(years)+'.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_TEST, Y_TEST)[1]))

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [377]:
years = 2010 

pol_2010_data = pd.read_csv("./Congressmen+Speech/POL_RES/polres_"+str(years)+".csv",encoding="euc-kr")

# 정제와 단어 토큰화
vocab = {} # 파이썬의 dictionary 자료형
sentences = []
stop_words = set(stopwords.words('english'))

pol_2010_data["textlens"] = 0

for te in range(0,len(pol_2010_data)):
    text = sent_tokenize(pol_2010_data["V2"][te])
    del text[0]
    pol_2010_data["textlens"][te] = len(text)
    for i in text:
        
        sentence = word_tokenize(i) # 단어 토큰화를 수행합니다.
        result = []

        for word in sentence: 
            word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
            if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
                if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                    result.append(word)
                    if word not in vocab:
                        vocab[word] = 0 
                    vocab[word] += 1
        sentences.append(result) 
        
vocab_sorted = sorted(vocab.items(), key = lambda x:x[1], reverse = True)

word_to_index = {}
i=0
for (word, frequency) in vocab_sorted :
    if frequency > 1 : # 
        i=i+1
        word_to_index[word] = i
        

vocab_size = 99999
words_frequency = [w for w,c in word_to_index.items() if c >= vocab_size + 1] 
for w in words_frequency:
    del word_to_index[w] # 해당 단어에 대한 인덱스 정보를 삭제
    
    
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################
##########################################################################################

loaded_model = load_model('best_model_'+str(years)+'.h5')
pathis = './CEO earnings call/'+str(years)+'_COMPANY/'
fn = os.listdir(pathis)

for f in fn:
    try:
        col_data = pd.read_csv(pathis+f,encoding="euc-kr")
    except:
        continue
    # 정제와 단어 토큰화
    vocab = {} # 파이썬의 dictionary 자료형
    sentences = []

    for te in range(0,len(col_data)):
        
        try:
            text = sent_tokenize(col_data["text"][te])
        except:
            continue
            
        for i in text:
            sentence = word_tokenize(i) # 단어 토큰화를 수행합니다.
            result = []

            for word in sentence: 
                word = word.lower() # 모든 단어를 소문자화하여 단어의 개수를 줄입니다.
                if word not in stop_words: # 단어 토큰화 된 결과에 대해서 불용어를 제거합니다.
                    if len(word) > 2: # 단어 길이가 2이하인 경우에 대하여 추가로 단어를 제거합니다.
                        result.append(word)
                        if word not in vocab:
                            vocab[word] = 0 
                        vocab[word] += 1
            sentences.append(result) 

    encoded = []
    for s in sentences:
        temp = []
        for w in s:
            try:
                temp.append(word_to_index[w])
            except KeyError:
                continue

        encoded.append(temp)

    val_0 = []
    val_1 = []
    for i in range(0,len(encoded)):
        if(len(encoded[i]) >= 10):
            aa = loaded_model.predict(encoded[i])

            rows = aa.shape[0]
            cols = aa.shape[1]
            for r in range(0,rows):

                if(aa[r][0] >= 0.8 or aa[r][1] >= 0.8): # 정치색이 강한 단어만 선택 (80%)
                    val_0.append(aa[r][0])
                    val_1.append(aa[r][1])   
                    
                    
    try:
        print("회사이름 : " + f)
        print("공화당적 발언 정도 : " + str(sum(val_0)/len(val_0)) )
        print("민주당적 발언 정도 : " + str(sum(val_1)/len(val_1)) )
    except:
        continue
                

화사이름 : 2020_ and welcome.csv
화사이름 : 2020_ Joe. Good afternoon.csv
화사이름 : 2020_ John. Good afternoon and welcome to our first quarter earnings call. Before we begin.csv
화사이름 : 2020_ Rob. Overall.csv
화사이름 : 2020_.csv
화사이름 : 2020_1-800-FLOWERS.COM, Inc..csv
공화당적 발언 정도 : 0.6477891047024864
민주당적 발언 정도 : 0.3522108921716953
화사이름 : 2020_10x Genomics, Inc..csv
공화당적 발언 정도 : 0.4956341708000277
민주당적 발언 정도 : 0.5043658202277942
화사이름 : 2020_111, Inc..csv
공화당적 발언 정도 : 0.5878028806411859
민주당적 발언 정도 : 0.4121971219175994
화사이름 : 2020_180 Degree Capital Corp.csv
공화당적 발언 정도 : 0.47394437130505795
민주당적 발언 정도 : 0.5260556239251778
화사이름 : 2020_1Life Healthcare, Inc..csv
공화당적 발언 정도 : 0.4644277523902635
민주당적 발언 정도 : 0.5355722412707344
화사이름 : 2020_21Vianet Group, Inc..csv
공화당적 발언 정도 : 0.5150176038344702
민주당적 발언 정도 : 0.48498239951829114


KeyboardInterrupt: 